In [4]:
import os
import json
import time
from IPython.display import Markdown, display
import requests
import textwrap
import boto3
from utils import read_file, save_file
from dotenv import load_dotenv

In [5]:
load_dotenv()
ab_paper = read_file('data/whitepaper/AB_2013-07_Model_Risk_Management_Guidance.md')
moody_paper = read_file('data/whitepaper/riskcalc-3.1-whitepaper.md')

In [6]:
br = boto3.client(service_name='bedrock')
model_summaries = br.list_foundation_models()['modelSummaries']
#print(json.dumps(model_summaries, indent=4))

In [10]:
cloude = br.get_foundation_model(modelIdentifier='anthropic.claude-3-haiku-20240307-v1:0')['modelDetails']
print(json.dumps(cloude, indent=4))

{
    "modelArn": "arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-haiku-20240307-v1:0",
    "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
    "modelName": "Claude 3 Haiku",
    "providerName": "Anthropic",
    "inputModalities": [
        "TEXT",
        "IMAGE"
    ],
    "outputModalities": [
        "TEXT"
    ],
    "responseStreamingSupported": true,
    "customizationsSupported": [],
    "inferenceTypesSupported": [
        "ON_DEMAND"
    ],
    "modelLifecycle": {
        "status": "ACTIVE"
    }
}


In [74]:
def get_document_analysis_claude(document, question, model='anthropic.claude-3-haiku-20240307-v1:0', temperature=0, tokens=3000, top_p=0.9, top_k=250):
    instructions = f"""As a model risk manager, please conduct a detailed model analysis. Think step by step.
You are provided with model whitepaper to perform the analysis.
You are provided with analysis objective in the user input.
Your response should use well formatted Markdown forman and must include:
1. Numbered list of all relevant sections in the whitepaper
2. Section with specific findings. The finding must include supporting references to the specific sections in the whitepaer. 
3. Section with a clear, evidence-based recommendations on whether the model should be adopted for usage, given the findings.
<whitepaper>
{document}
</whitepaper>
<example>
### Thought Process
1. Find relevant sections in the paper
2. Identity analysis tasks
3. ...
### Findings
1. The RiskCalc v3.1 model incorporates forward-looking market information through the distance-to-default factor (Secton 2.1), which captures systematic risk and industry trends not fully reflected in the financial statements alone.
</example>
"""
    
    brt = boto3.client(service_name='bedrock-runtime')
    
    body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "system": instructions,
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": question
                }
            ]
        }
    ],
    "max_tokens": tokens,
    "temperature": temperature,
    "top_p": top_p,
    "top_k": top_k
    })

    accept = 'application/json'
    contentType = 'application/json'

    response = brt.invoke_model(body=body, modelId=model, accept=accept, contentType=contentType)

    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0]['text']

In [75]:
qq = ['Identify any specific limitations and model usage risk in stagflation environment',
      'Indentify any specific limitations and model usage risks in hyper-inflation scenario']

model = 'anthropic.claude-3-haiku-20240307-v1:0'
#model = 'anthropic.claude-3-sonnet-20240229-v1:0'
for i, q in enumerate(qq):
    content = get_document_analysis_claude(moody_paper, q, model=model, tokens=4096)
    title = (f"## {q.capitalize()}")
    display(Markdown(title))
    display(Markdown(content))
    #save_file(f"reports/moody-risk-calc-analysis-cloude-21-{i+1}.md", f"{title}\n{content}")


## Identify any specific limitations and model usage risk in stagflation environment

Based on my review of the whitepaper, I did not find any explicit discussion of the model's limitations or risks in a stagflationary environment specifically. However, there are a few relevant points that can be inferred:

1. The model incorporates forward-looking market information through the distance-to-default factor, which captures systematic risk and industry trends (Section 3.2). This suggests that in an environment of stagflation, where economic growth stagnates while inflation rises, the model may be able to capture the increased systematic risk through changes in this market-based factor.

2. The model allows for stress testing of firms under different economic scenarios by holding the firm's financial statements constant while varying the systematic risk factors (Section 2.3, Figure 1). This capability could potentially be used to assess a firm's risk under a stagflationary scenario.

3. However, the paper notes that the relationship between default rates and some macroeconomic variables like interest rates and unemployment is inconsistent over time (Section 3.2). This suggests that in an abnormal economic environment like stagflation, the model's predictive power may be impacted if the relationships between firm financials/market data and defaults change significantly.

4. The model is re-estimated and calibrated periodically using an expanding data set that covers different economic cycles (Section 2.2). This process of continual re-estimation may help account for changing environments, but is not explicitly discussed for stagflationary periods.

So in summary, while not directly addressed, the combination of forward-looking market factors and stress testing capabilities could allow the model to account for stagflationary impacts to some degree. However, the potential for parameter instability in such abnormal environments, and the need for re-estimation using data from those periods, represents a risk limitation that is not explicitly analyzed in the whitepaper.

## Indentify any specific limitations and model usage risks in hyper-inflation scenario

The whitepaper does not explicitly discuss limitations or risks of using the RiskCalc v3.1 model in a hyper-inflation scenario. However, some potential limitations and risks can be inferred from the model details provided:

1. Financial ratios based on historical financial statements may not accurately reflect a firm's current risk profile during periods of high inflation, as the accounting data lags behind rapidly changing economic conditions.

2. The distance-to-default factor, which captures forward-looking market information, may not fully incorporate expectations about hyper-inflationary conditions if such scenarios are not priced into equity markets.

3. The model parameters and transformations are estimated based on historical data that may not cover periods of extreme inflation. This could lead to poor model calibration and inaccurate default probability estimates during hyper-inflation.

4. Rapidly changing inflation rates can impact the stability and interpretation of financial ratios used as model inputs, potentially reducing the model's predictive power.

5. The model may need to be re-estimated and recalibrated using data from periods of high inflation to maintain accuracy under such economic conditions.

While not explicitly stated, the whitepaper emphasizes the importance of continuously validating and updating the model as new data becomes available to account for changing economic conditions. This suggests that in a hyper-inflationary scenario, the model may need to be closely monitored, updated, and potentially re-estimated to maintain its predictive performance.

In [35]:
qq = ["Generate a list of the model analysis objectives using provided model whitepaper",
      "Generate a list of objecttives for assessing the model peformance in different economic environments usingf providedl model whitepaper"]

for i, q in enumerate(qq):
    content = get_document_analysis_claude(moody_paper, q, model=model, tokens=4096)
    title = (f"## {q.capitalize()}")
    display(Markdown(title))
    display(Markdown(content))
    save_file(f"reports/moody-riskcalc-objectives-claude-{i+1}.md", f"{title}\n{content}")

## Generate a list of the model analysis objectives using provided model whitepaper

Here is a detailed model analysis focusing on the objectives using the provided model whitepaper:

## Model Analysis Objectives

Based on the whitepaper, the key objectives of the RiskCalc v3.1 model analysis are:

**1. Assess the strategic assets and innovations of the RiskCalc v3.1 model:**
- Evaluate the benefits of the expanded data pool for predictions, including the size and quality improvements to the Credit Research Database.
- Examine how the model supports regulatory requirements under the New Basel Capital Accord, such as providing consistent risk estimates, forward-looking risk ratings, stress testing default probabilities, and robust model validation.

**2. Understand the model framework and key features:**
- Examine the Financial Statement Only (FSO) mode and how it selects a limited set of financial ratios to avoid overfitting.
- Analyze the complete RiskCalc v3.1 model, including how it blends firm-specific financial data with forward-looking market information through the distance-to-default measure.
- Evaluate the importance of controlling for industry variation in the model.

**3. Assess the model validation and performance:**
- Review the out-of-sample and out-of-time testing approaches, including K-fold analysis and walk-forward testing, to evaluate model power and calibration.
- Examine the model's performance over the credit cycle and compare it to alternative models like the Private Firm Model and Z-score.

**4. Evaluate the economic value of the RiskCalc v3.1 model:**
- Analyze the potential profit and loss benefits for lenders using the RiskCalc v3.1 model compared to alternative models.

## Key Findings from the Model Analysis

1. **Strategic Assets and Innovations:**
   - The RiskCalc v3.1 model is built on the world's largest and cleanest private company default database, the Moody's KMV Credit Research Database, which has over 6.5 million financial statements and 97,000 default events globally.
   - The model is designed to meet the requirements of the New Basel Capital Accord, including providing consistent risk estimates, forward-looking risk ratings, the ability to stress test default probabilities, and robust model validation.

2. **Model Framework and Key Features:**
   - The Financial Statement Only (FSO) mode selects a limited set of financial ratios to capture profitability, leverage, debt coverage, growth, liquidity, activity, and size, avoiding overfitting by using non-parametric transformations.
   - The complete RiskCalc v3.1 model blends firm-specific financial data with forward-looking market information through the distance-to-default measure, which captures systematic industry and economic trends.
   - Controlling for industry variation is crucial, as it allows the model to account for differences in default rates and the interpretation of financial ratios across sectors.

3. **Model Validation and Performance:**
   - Rigorous out-of-sample and out-of-time testing, including K-fold analysis and walk-forward testing, demonstrates the model's strong predictive power and calibration, outperforming alternatives like the Private Firm Model and Z-score.
   - The model maintains its performance over the credit cycle, with the complete RiskCalc v3.1 model showing significantly higher accuracy ratios compared to previous versions and other models.

4. **Economic Value:**
   - Analyses suggest the RiskCalc v3.1 model can provide substantial economic benefits for lenders, potentially increasing the profitability of a loan portfolio by up to 25 basis points compared to alternatives.
   - In a competitive environment, a medium-sized bank using the RiskCalc v3.1 model could see profits increase by more than $10 million on average compared to a competitor using a weaker model like Z-score.

## Recommendation

Based on the comprehensive analysis of the RiskCalc v3.1 model, I would recommend adopting this model for usage. The model demonstrates significant improvements in predictive power, calibration, and economic value compared to previous versions and alternative models. The rigorous validation testing, adherence to regulatory requirements, and the model's ability to capture both firm-specific and systematic risk factors make it a compelling choice for lenders seeking to improve their credit risk assessment and portfolio management. The potential profit and loss benefits of using the RiskCalc v3.1 model are substantial and could provide a meaningful competitive advantage in the market.

## Generate a list of objecttives for assessing the model peformance in different economic environments usingf providedl model whitepaper

Here is a detailed model analysis focusing on assessing the model performance in different economic environments using the provided model whitepaper:

**Bullet points highlighting specific analysis topics:**

- The RiskCalc v3.1 model is designed to meet the requirements of the New Basel Capital Accords, including extensive documentation, validation, and testing. This allows the model to provide consistent risk estimates, forward-looking risk ratings, and the ability to stress test default probabilities under different economic conditions (Section 2.3).
- The model's performance is validated using rigorous out-of-sample and out-of-time testing, including K-fold analysis and walk-forward analysis. These tests demonstrate the model's stability and superior predictive power compared to previous versions and other models like the Private Firm Model and Z-score (Section 4).
- The model's performance is shown to be robust across the credit cycle, maintaining high accuracy ratios even during periods of high default activity (Section 4.4).
- The expanded data pool used to develop the RiskCalc v3.1 model, including data from the 2000-2002 period of high default rates, allows the model to be calibrated to a wide range of general credit cycle conditions and to stress test the impact of a changing economy on default likelihoods (Section 2.2).
- The incorporation of industry-specific information and market-based factors (distance-to-default) into the model allows it to better capture systematic, sector-based risks that may not be fully reflected in financial statements alone. This improves the model's performance, particularly over shorter horizons when market information leads the accounting data (Sections 3.2 and 3.3).

**Recommendation:**

Based on the detailed analysis provided in the whitepaper, the RiskCalc v3.1 model appears to be a robust and high-performing model for assessing the credit risk of private, middle-market companies. The model's design to meet regulatory requirements, its superior predictive power demonstrated through rigorous validation, and its ability to capture systematic and idiosyncratic risks across different economic environments make it a strong candidate for adoption.

The whitepaper provides compelling evidence that the RiskCalc v3.1 model outperforms previous versions as well as other widely used models like the Private Firm Model and Z-score. The model's stress testing capabilities and responsiveness to changing market conditions are also valuable features that allow users to assess credit risk under a variety of economic scenarios.

Overall, the detailed analysis presented in the whitepaper suggests that the RiskCalc v3.1 model should be strongly considered for adoption, as it appears to be a significant improvement over previous models and well-suited for assessing private firm credit risk in different economic environments.